<a href="https://colab.research.google.com/github/AndryADSM/Asset-Analyzer/blob/main/asset_analyzer_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asset Analyzer
Module with the class PriceData  
My Website: https://aadsm2355.wixsite.com/andryadsm  

# Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# Class PriceData

In [2]:
class PriceData(pd.DataFrame):
    """
        Downloads price and volume data for each ticker from Yahoo Finance.

        Params
        ---
        - tickers: list
            List of tickers to download.
        - start: str
            The start date formatted as 'YYYY-MM-DD', inclusive.
            If None, defaults to the maximum available time.
        - end: str
            The end date formatted as 'YYYY-MM-DD', exclusive.
            If None, defaults to the
        - interval: str='1d'
            Valid intervals:
                '1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo' or '3mo'.
            intraday data cannot extend last 60 days,
            except '60m' or '1h', up to last 730 days (1.5 years)
            and '1m', up to last 7 days (1 week)
        - actions: bool=False
            Download dividend & stock splits data.
        - auto_adjust: bool=False
            Adjusts all OHLC automatically.

        Methods and Subclasses
        ---
        class PctReturns
            class Insights
                top_results()
            insights()
            plot_dist_ret()
        pct_returns()
        pct_returns_daily()
        -
        class NormPrices
            plot_norm_moves()
            plot_norm_vol()
        normalize_values()
        -
        drop_columns()
        from_datetime()
        -
        plot_price()
        plot_volume()
        plot_price_vol()
        -
        sharpe_ratio()
        correlations()
        calc_rsi()
        calc_historical_volatility()
        calc_bollinger_bands()
        calc_macd()
        monthly_returns()
        hourly_returns()

        Example use
        ---
        `tickers = ['SPY', 'AAPL', 'TSLA', 'KO']`
        `data = aa.PriceData(tickers, '2023-01-01', '2024-01-01', '1d')`
    """

    ################################
    #### Dunders
    ################################

    def __init__(
            self,
            tickers:     list[str],
            start:       str,
            end:         str,
            interval:    str  = '1d',
            *,
            actions:     bool = False,
            auto_adjust: bool = False
    ) -> None:

        data = yf.download(tickers,
                           start=start,
                           end=end,
                           group_by='ticker',
                           interval=interval,
                           actions=actions,
                           auto_adjust=auto_adjust)

        if len(tickers) == 1:
            data.columns = pd.MultiIndex.from_product([tickers, data.columns],
                                                      names=['Ticker', 'Price'])

        super().__init__(data[tickers])

        self.tickers     = str(tickers)
        self._start      = start
        self._end        = end
        self.start_dt    = data.index[0]
        self.end_dt      = data.index[-1]
        self.interval    = interval
        self.actions     = actions
        self.auto_adjust = auto_adjust

    def __repr__(self):
        "Return string of code used to build the instance."
        return f"PriceData({self.tickers}, '{self._start}', '{self._end}', '{self.interval}', actions={self.actions}, auto_adjust={self.auto_adjust})"

    def __str__(self):
        "Return string with information of the instance"
        return f'Price data. Tickers {self.tickers}, from {self.start_dt} to {self.end_dt}, interval of {self.interval}'

    ################################
    #### Returns
    ## class PctReturns
    ##     class Insights
    ##         top_results()
    ##     insights()
    ##     plot_dist_ret()
    ## pct_returns()
    ## pct_returns_daily()
    ################################

    class PctReturns(pd.DataFrame):
        """
            Subclass of PriceData

            Methods and Subclasses
            ---
            class Insights
                top_results()
            insights()
            plot_dist_ret()
        """

        def __init__(self, data) -> None:
            super().__init__(data)

        ################################
        #### Insights
        ## class Insights
        ##     top_results()
        ## insights()
        ## plot_dist_ret()
        ################################

        class Insights(pd.DataFrame):
            """
                Subclass of PctReturns

                Method
                ---
                top_results()
            """

            def __init__(self, data) -> None:
                super().__init__(data)

            def top_results(
                    self,
                    *,
                    top: int = 3,
                    bot: int = 3,
                    round_val: int = 6
            ) -> None:
                """
                    Displays the top and bottom results of the insights data.
                    #### return None

                    Params
                    ---
                    - top: int=3
                        Top n to display.
                    - bot: int=3
                        Bottom n to display.
                    - round_val: int=6
                        Number of decimals to be rounded to.

                    Example use
                    ---
                    `data_ins.top_results(top=2, bot=1, round_val=3)`
                """

                for i in self.index:
                    display(
                        pd.DataFrame(
                            [self.loc[i].nlargest(top).round(round_val),
                            self.loc[i].nsmallest(bot).round(round_val).sort_values(ascending=False)],
                            index=[f'Top {i[0]}, {i[1]}', f'Bottom {i[0]}, {i[1]}']
                        )
                    )

        def insights(
                self,
                *,
                volatility: bool = True,
                mean:       bool = True,
                mean_abs:   bool = True,
                order_by:   str  = 'ins'
        ) -> Insights:
            """
                Gets insights like volatility, mean and mean of absolute values
                about the returns data.
                #### return Insights

                Params
                ---
                - volatility: bool=True
                    If to include the volatility measure.
                - mean: bool=True
                    If to include the mean measure.
                - mean_abs: bool=True
                    If to include the mean of the absolute values measure.
                - order_by: str='ins'
                    If 'ins', order the index by insights,
                    if 'ret', order by return.

                Example use
                ---
                `data_ins = data_ret.insights(order_by='ins')`
            """

            data_ins = pd.DataFrame(
                columns=pd.MultiIndex.from_product([[], []], names=['Insight', 'Return'])
            )

            tks = self.columns.get_level_values(0).unique()
            ret = self.columns.get_level_values(1).unique()

            for col in ret:
                if volatility:
                    data_ins['Volatility', f'{col}'] = {tk: (self[tk, col].std()) for tk in tks}
                if mean:
                    data_ins['Mean', f'{col}'] = {tk: self[tk, col].mean() for tk in tks}
                if mean_abs:
                    data_ins['Mean_abs', f'{col}'] = {tk: self[tk, col].abs().mean() for tk in tks}

            data_ins = data_ins.T
            data_ins = data_ins.loc[data_ins.index.get_level_values(0).unique()]

            if order_by == 'ret':
                data_ins = data_ins.swaplevel('Return', 'Insight').loc[ret]

            df = PriceData.PctReturns.Insights(data_ins)

            return df

        def plot_dist_ret(
                self,
                tickers:   list[str] | None  = None,
                start:     str | None        = None,
                end:       str | None        = None,
                *,
                style:     str               = 'dark_background',
                figsize:   tuple[int] | None = None,
                use_ret:   str               = 'Daily',
                colors:    list[str] | None  = None,
                hist:      bool              = True,
                kde:       bool              = True,
                legendloc: str | None        = None
        ) -> None:
            """
                Plots the distribution of the returns of the given period.
                #### return None

                Params
                ---
                - tickers: list[str] | None= None
                    The tickers to be plotted, maximum of three,
                    if None, defaults to plot the first three tickers.
                - start: str | None=None
                    The start date formatted as 'YYYY-MM-DD', inclusive.
                - end: str | None=None
                    The end date formatted as 'YYYY-MM-DD', inclusive.
                - style: str='dark_background'
                    Style of the plot, check available styles with `plt.style.available`
                - figsize: tuple[int] | None=None
                    Width and height of the plot,
                    if None, defaults to (16, 6)
                - use_ret: str='Daily'
                    The return column to be used for plotting the price.
                - colors: list[str] | None=None
                    List with the color of each element in order,
                    if None, defaults to a palette.
                - hist: bool=True
                    If to show histogram plot.
                - kde: bool=True
                    If to show kernel density estimate plot.
                - legendloc: str | None=None
                    Location of the legend,
                    if None, defaults to the best location.

                Example use
                ---
                `data_ret.plot_dist_ret(['SPY', 'AAPL'], colors=['w', 'r'], use_ret='Intraday')`
            """

            if tickers is None:
                tickers = self.columns.get_level_values(0).unique()[:3]
            else:
                tickers = tickers[:3]
            data_to_plot = self.loc[start:end][tickers]

            if colors is None:
                colors = ['r', 'b', 'g']
            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'Distribution of {use_ret} Return', size=18)

                for ticker, color in zip(tickers, colors):
                    if hist:
                        sns.histplot(data_to_plot[ticker,  use_ret],
                                    bins=50, color=color,
                                    edgecolor=None,
                                    alpha=0.5,
                                    kde=False,
                                    stat="density",
                                    zorder=2,
                                    label=ticker)

                    if kde:
                        sns.kdeplot(data_to_plot[ticker,  use_ret],
                                    color=color,
                                    zorder=2,
                                    label=ticker)

                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.legend(loc=legendloc)
                plt.xlabel(f'{use_ret} Return')
                plt.ylabel(f'Density')
                plt.show()

    def pct_returns(
            self,
            values: dict | None = None,
            *,
            round_val: int  = 6,
            as_pct:    bool = True
    ) -> PctReturns:
        """
            Gets percentage return for the specified time frames.
            #### return PctReturns

            Params
            ---
            - values: dict | None=None
                Takes dictionary in the form {str: (str, str, int)}
                notation {'Name_of_return': ('After_price', 'Before_price', shift_of_before_price)}
                if None, defaults to {'Daily': ('Close', 'Close', 1)}
            - round_val: int=6
                Number of decimals to be rounded to.
            - as_pct: bool=True
                If True, values are expressed as percentages (e.g. 1.23 (%)),
                if False, as decimals (e.g. 0.0123).

            Example use
            ---
            `data_ret = data.pct_returns(values={'O_to_C': ('Close', 'Open' , 0), 'HL_diff': ('High' , 'Low'  , 0)}, round_val=2)`
        """

        if values is None:
            values = {'Daily': ('Close', 'Close', 1)}

        df = pd.DataFrame(index=self.index,
                          columns=pd.MultiIndex.from_product([[], []],
                                                             names=['Ticker', 'TimeFrame']))

        for tk in self.columns.get_level_values(0).unique():
            # General formula
            for key, val in values.items():
                df[tk, f'{key}'] = (self[tk, val[0]] - self[tk, val[1]].shift(val[2])) \
                                   / self[tk, val[1]].shift(val[2])

        # Turn percentage into decimal
        if as_pct:
            df *= 100

        df = df.round(round_val)

        df = PriceData.PctReturns(df)

        return df

    def pct_returns_daily(
            self,
            *,
            round_val: int  = 6,
            as_pct:    bool = True
    ) -> PctReturns:
        """
            Gets percentage return for different time frames for daily price data.
            #### return PctReturns

            Params
            ---
            - round_val: int=6
                Number of decimals to be rounded to.
            - as_pct: bool=True
                If True, values are expressed as percentages (e.g. 1.23 (%)),
                if False, as decimals (e.g. 0.0123).

            Example use
            ---
            `data_ret_d = data.pct_returns_daily()`
        """

        values = {
            'Intraday':  ('Close', 'Open' , 0 ),
            'Overnight': ('Open' , 'Close', 1 ),
            'Daily':     ('Close', 'Close', 1 ),
            '1week':     ('Close', 'Close', 5 ),
            '2week':     ('Close', 'Close', 10),
            '3week':     ('Close', 'Close', 15),
            '4week':     ('Close', 'Close', 20)
        }
        df = self.pct_returns(values=values, round_val=round_val, as_pct=as_pct)
        return df

    ################################
    #### Normalized Valules
    ## class NormPrices
    ##     plot_norm_moves()
    ##     plot_norm_vol()
    ## normalize_values()
    ################################

    class NormPrices(pd.DataFrame):
        """
            Subclass of PriceData

            Methods
            ---
            plot_norm_moves()
            plot_norm_vol()
        """

        def __init__(self, data) -> None:
            super().__init__(data)

        def plot_norm_moves(
                self,
                tickers:   list[str] | None  = None,
                start:     str | None        = None,
                end:       str | None        = None,
                *,
                style:     str               = 'dark_background',
                figsize:   tuple[int] | None = None,
                use_price: str               = 'Close_norm',
                colors:    list[str] | None  = None,
                legendloc: str | None        = None
        ) -> None:
            """
                Plots movement of normalized prices,
                compare movements between assets by giving more than one ticker.
                #### return None

                Params
                ---
                - ticker: list[str] | None=None
                    The tickers to plot,
                    if None, default to plot all.
                - start: str | None=None
                    The start date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
                - end: str | None=None
                    The end date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
                - style: str='dark_background'
                    Style of the plot, check available styles with `plt.style.available`
                - figsize: tuple[int] | None=None
                    Width and height of the plot,
                    if None, defaults to (16, 6)
                - use_price: str='Close_norm'
                    The normalized price column to be used for plotting the price.
                - colors: list[str] | None=None
                    List with the color of each line in order,
                    if None, defaults to a palette.
                - legendloc: str | None=None
                    Location of the legend,
                    if None, defaults to the best location.

                Example use
                ---
                `data_norm.plot_norm_moves(['SPY', 'AAPL'], colors=['w', 'r'])`
            """

            # Prepare data to be plotted
            if tickers is None:
                tickers = self.columns.get_level_values(0).unique()
            date_format = '%Y-%m-%d'
            if self.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            data_to_plot = self.loc[start:end][tickers]
            data_to_plot.index = data_to_plot.index.strftime(date_format)

            if colors is None:
                colors = ['r', 'b', 'g', 'y', 'm', 'c']
            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title('Movement Normalized', size=18)

                for i, tk in enumerate(tickers):
                    plt.plot(data_to_plot[tk, use_price],
                            color=colors[i % len(colors)],
                            alpha=1,
                            label=tk)

                plt.grid(which='major', color='grey', linestyle=':')
                plt.xticks(np.arange(len(data_to_plot),
                                    step=max(len(data_to_plot) // 10, 1)),
                        size=7)
                plt.legend(loc=legendloc)
                plt.xlabel('Date')
                plt.ylabel(f'{use_price.split("_")[0]} Price Normalized')
                plt.show()

        def plot_norm_vol(
                self,
                tickers:   list[str] | None  = None,
                start:     str | None        = None,
                end:       str | None        = None,
                *,
                style:     str               = 'dark_background',
                figsize:   tuple[int] | None = None,
                colors:    list[str] | None  = None,
                legendloc: str | None        = None
        ) -> None:
            """
                Plots movement of the normalized volume,
                compare movements between assets by giving more than one ticker.
                #### return None

                Params
                ---
                - tickers: list[str] | None= None
                    The tickers to be plotted, maximum of three,
                    if None, defaults to plot the first three tickers.
                - start: str | None=None
                    The start date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
                - end: str | None=None
                    The end date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
                - style: str='dark_background'
                    Style of the plot, check available styles with `plt.style.available`
                - figsize: tuple[int] | None=None
                    Width and height of the plot,
                    if None, defaults to (16, 6)
                - colors: list[str] | None=None
                    List with the color of each line in order,
                    if None, defaults to a palette.
                - legendloc: str | None=None
                    Location of the legend,
                    if None, defaults to the best location.

                Example use
                ---
                `data.plot_norm_vol(['SPY', 'AAPL'], colors=['w', 'r'])`
            """

            # Prepare data to be plotted
            if tickers is None:
                tickers = self.columns.get_level_values(0).unique()[:3]
            date_format = '%Y-%m-%d'
            if self.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            data_to_plot = self.loc[start:end][tickers]
            data_to_plot.index = data_to_plot.index.strftime(date_format)

            if colors is None:
                colors = ['r', 'b', 'g']
            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title('Volume Normalized', size=18)

                for ticker, color in zip(tickers, colors):
                    plt.plot(data_to_plot[ticker, 'Volume_norm'],
                            label=ticker,
                            color=color)

                plt.grid(which='major', color='grey', linestyle=':')
                plt.xticks(np.arange(len(data_to_plot),
                                    step=max(len(data_to_plot) // 10, 1)),
                        size=7)
                plt.legend(loc=legendloc)
                plt.xlabel('Date')
                plt.ylabel('Movement')
                plt.show()

    def normalize_values(
            self,
            *,
            volume: bool = True
    ) -> NormPrices:
        """
            Gets normalized price and volume data.
            #### return NormPrices
            Params
            ---
            - volume: bool=True
                If to include normalized volume data

            Example use
            ---
            `data_norm = data.normalize_values()`
        """

        data_norm = pd.DataFrame(columns=pd.MultiIndex.from_product([[], []],
                                                                    names=['Ticker', 'Price']))

        tks = [tk for tk in self.columns.get_level_values(0).unique() if tk != 'Datetime']
        prices = [price for price in self.columns.get_level_values(1).unique() \
                if price in ['Open', 'High', 'Low', 'Close', 'Adj Close']]

        # Calculate min and max price across all prices for each ticker
        min_price = {tk: self[tk][prices].min().min() for tk in tks}
        max_price = {tk: self[tk][prices].max().max() for tk in tks}

        # Normalize between 0 and 1
        for tk in tks:
            for price in prices:
                data_norm[tk, f'{price}_norm']  = (self[tk, price]  - min_price[tk]) \
                                                / (max_price[tk] - min_price[tk])

            if volume and 'Volume' in self.columns.get_level_values(1).unique():
                vol = self[tk, 'Volume']
                data_norm[tk, 'Volume_norm'] = (vol - vol.min()) \
                                            / (vol.max() - vol.min())

        return PriceData.NormPrices(data_norm)

    ################################
    #### Drop Columns, Get Date Values
    ## drop_columns()
    ## from_datetime()
    ################################

    def drop_columns(
            self,
            cols: str | list[str]
    ):
        """
            Drops all the columns specified out of the second level.
            Similar to using the pandas drop() method.
            #### return PriceData

            Params
            ---
            - cols: str | list[str]
                Labels of columns to be dropped.

            Example use
            ---
            `data.drop_columns('Adj Close')`
        """

        # To be able to input a string
        if type(cols) is str:
            cols = [cols]

        self.drop(columns=cols, inplace=True, level=1, errors='ignore')
        return self

    def from_datetime(
            self,
            get: str,
            *,
            add_col: bool = False
    ) -> pd.DataFrame:
        """
            Get a DataFrame with the datetime value gotten from the index.
            Can concatenate with the source DataFrame.
            #### return pd.DataFrame or PriceData

            Params
            ---
            - get: str
                The value to get from the datetime index.
                Valid inputs are:
                    'minute', 'hour', 'weekday', 'day_name', 'day', 'month', 'month_name', 'quarter' or 'year'
            - add_col: bool=False
                If to add a Datetime column to the source data.

            Example use
            ---
            `data.from_datetime('weekday', add_col=True)`
        """

        # Check if the input for get is valid
        inputs = ['minute', 'hour', 'weekday', 'day_name',
                  'day', 'month', 'month_name', 'quarter', 'year']

        if get not in inputs:
            raise ValueError(f'Invalid input, get=\'{get}\', valid inputs are: {inputs}')

        # Check if the column already exist
        if add_col and f'{get.capitalize()}' in self.columns.get_level_values(1).unique():
            raise Exception(f'Already a column named \'{get}\'.')

        dt = self.index.to_frame().iloc[:,0].dt

        # Get the actual datetime
        if 'name' in get:
            dt = eval(f'dt.{get}()')
        else:
            dt = eval(f'dt.{get}')

        if add_col:
            self[('Datetime', f'{get.capitalize()}')] = dt
            return self
        else:
            dt = dt.to_frame()
            dt.columns = [f'{get.capitalize()}']
            return dt

    ################################
    #### Plots
    ## plot_price()
    ## plot_volume()
    ## plot_price_vol()
    ################################

    def plot_price(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            style:     str                 = 'dark_background',
            figsize:   tuple[float] | None = None,
            use_price: str                 = 'Close',
            color:     str                 = 'c',
            mas:       list[int] | bool    = True,
            mas_color: list[str] | None    = None,
            legendloc: str | None          = None
    ) -> None:
        """
            Plots the price and moving averages of a specific ticker.
            #### return None

            Params
            ---
            - ticker: str
                The ticker to plot.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - use_price: str='Close'
                The price column to be used for plotting the price.
            - color: str='cyan'
                Color of the price line.
            - mas: list[str] | None=None
                Length of price moving averages, as many as wished,
                if None, defaults to 20, 40, 100 and 200
            - mas_color: list[str] | None=None
                Color of each of the price moving averages in order,
                if None, defaults to a palette.
            - legendloc: str | None=None
                Location of the legend,
                if None, defaults to the best location.

            Example use
            ---
            `data.plot_price('SPY', color='w', mas=[20, 50, 100, 150])`
        """
        # Prepare data to be plotted
        date_format = '%Y-%m-%d'
        if self.index.name == 'Datetime':
            date_format = '%Y-%m-%d %H:%M'
        data_to_plot = self.loc[start:end][ticker]
        data_to_plot.index = data_to_plot.index.strftime(date_format)
        data_ma = self[ticker]
        data_ma.index = data_ma.index.strftime(date_format)

        # Plot
        with plt.style.context(style):
            if figsize is None:
                figsize = (16, 6)

            plt.figure(figsize=figsize)
            plt.title(f'{ticker} Price', size=18)

            # Price
            plt.plot(data_to_plot[use_price],
                    color=color,
                    linewidth=2,
                    zorder=20,
                    label=f'{use_price} Price')

            # Price Moving Averages
            if mas is not False:
                if mas is True:
                    mas = [20, 40, 100, 200]
                if mas_color is None:
                    mas_color = ['y', 'r', 'g', 'm']

                for m, c, z in zip(mas,
                                range(len(mas_color)+100),
                                range(len(mas)-1, -1, -1)):
                    plt.plot(data_ma[use_price].rolling(m).mean().loc[start:end],
                            color=mas_color[c % len(mas_color)],
                            zorder=z,
                            label=f'{m}MA')

            plt.grid(which='major', color='grey', linestyle=':')
            plt.xticks(np.arange(len(data_to_plot),
                                step=max(len(data_to_plot) // 10, 1)),
                    size=7)
            plt.legend(loc=legendloc)
            plt.xlabel('Date')
            plt.ylabel('Price ($)')
            plt.show()

    def plot_volume(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            style:     str                 = 'dark_background',
            figsize:   tuple[float] | None = None,
            bar_color: str | None          = None,
            ma:        int | bool          = True,
            ma_color:  str                 = 'y',
            legendloc: str | None          = None
    ) -> None:
        """
            Plots volume and a moving average of a specific ticker.
            #### return None

            Params
            ---
            - ticker: str
                The ticker to plot.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - bar_color: str=None
                Color of the volume bars,
                if None, defaults to color based on the movement of each interval.
            - ma: int | bool=True
                Length of the volume moving average, only one.
                If False, do not show moving average,
                if True, defaults to a 20 moving average.
            - ma_color: str='orange'
                Color of the volume moving average line.
            - legendloc: str | None=None
                Location of the legend,
                if None, defaults to the best location.

            Example use
            ---
            `data.plot_volume('SPY', ma=50)`
        """

        # Prepare data to be plotted
        date_format = '%Y-%m-%d'
        if self.index.name == 'Datetime':
            date_format = '%Y-%m-%d %H:%M'
        data_to_plot = self.loc[start:end][ticker]
        data_to_plot.index = data_to_plot.index.strftime(date_format)
        data_ma = self[ticker]
        data_ma.index = data_ma.index.strftime(date_format)

        # Plot
        with plt.style.context(style):
            if figsize is None:
                figsize = (16, 6)

            plt.figure(figsize=figsize)
            plt.title(f'{ticker} Volume', size=18)

            # Color vol bars based on if the price went up or down in that interval
            if bar_color is None:
                period_ret = (data_ma['Close'] - data_ma['Open']) \
                            / data_ma['Open'] * 100
                colors = ['r' if ret < 0 else 'g' for ret in period_ret]
                bar_color = pd.Series(colors,
                                    index=period_ret.index
                                    ).loc[start:end]

            # Plot volume bars
            plt.bar(x=data_to_plot.index,
                    height=data_to_plot['Volume'],
                    color=bar_color,
                    edgecolor=bar_color,
                    label='Volume')

            # Volume Moving Average
            if ma is True:
                ma = 20
            if ma is not False:
                plt.plot(data_ma['Volume'].rolling(ma).mean().loc[start:end],
                        color=ma_color,
                        label=f'Vol {ma}MA')

            plt.grid(which='major', color='grey', linestyle=':')
            plt.xticks(np.arange(len(data_to_plot),
                                step=max(len(data_to_plot) // 10, 1)),
                    size=7)
            plt.legend(loc=legendloc)
            plt.xlabel('Date')
            plt.ylabel('Volume')

            plt.show()

    def plot_price_vol(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            style:           str                 = 'dark_background',
            figsize:         tuple[float] | None = None,
            use_price:       str                 = 'Close',
            price_color:     str                 = 'c',
            price_mas:       list[int] | bool    = True,
            price_mas_color: list[str] | None    = None,
            vol_bar_color:   str | None          = None,
            volume_ma:       int | bool          = True,
            vol_ma_color:    str                 = 'y',
            legendloc:       str | None          = None
    ) -> None:
        """
            Plots the price, volume and moving averages of a specific ticker.
            #### return None

            Params
            ---
            - ticker: str
                The ticker to plot.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD' or 'YYYY-MM-DD hh:mm', inclusive.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - use_price: str='Close'
                The price column to be used for plotting the price.
            - price_color: str='cyan'
                Color of the price line.
            - price_mas: list[str] | None=None
                Length of price moving averages, as many as wished,
                if None, defaults to 20, 40, 100 and 200
            - price_mas_color: list[str] | None=None
                Color of each of the price moving averages in order,
                if None, defaults to a palette.
            - vol_bar_color: str=None
                Color of the volume bars,
                if None, defaults to color based on the movement of each interval.
            - volume_ma: int | bool=True
                Length of the volume moving average, only one.
                If False, do not show moving average,
                if True, defaults to a 20 moving average.
            - vol_ma_color: str='orange'
                Color of the volume moving average line.
            - legendloc: str | None=None
                Location of the legend,
                if None, defaults to the best location.

            Example use
            ---
            `data.plot_price_vol('SPY', price_color='w', price_mas=[20, 50, 100, 150])`
        """

            # Prepare data to be plotted
        date_format = '%Y-%m-%d'
        if data.index.name == 'Datetime':
            date_format = '%Y-%m-%d %H:%M'
        data_to_plot = data.loc[start:end][ticker]
        data_to_plot.index = data_to_plot.index.strftime(date_format)
        data_ma = data[ticker]
        data_ma.index = data_ma.index.strftime(date_format)

        # Plot
        with plt.style.context(style):
            if figsize is None:
                figsize = (16, 6)

            fig, axs = plt.subplots(2,
                                    figsize=(figsize),
                                    height_ratios=(5, 2),
                                    sharex=True)
            plt.suptitle(f'{ticker} Price and Volume', size=18)
            plt.subplots_adjust(hspace=0.1)

            # Price
            axs[0].plot(data_to_plot[use_price],
                        color=price_color,
                        linewidth=2,
                        zorder=20,
                        label=f'{use_price} Price')

            # Price Moving Averages
            if price_mas is not False:
                if price_mas is True:
                    price_mas = [20, 40, 100, 200]
                if price_mas_color is None:
                    price_mas_color = ['y', 'r', 'g', 'm']

                for m, c, z in zip(price_mas,
                                range(len(price_mas_color)+100),
                                range(len(price_mas)-1, -1, -1)):
                    axs[0].plot(data_ma[use_price].rolling(m).mean().loc[start:end],
                                color=price_mas_color[c % len(price_mas_color)],
                                zorder=z,
                                label=f'{m}MA')

            axs[0].grid(which='major', color='grey', linestyle=':')
            axs[0].set_ylabel('Price ($)')
            axs[0].legend(loc=legendloc)

            # Volume
            # Color vol bars based on if the price went up or down in that interval
            if vol_bar_color is None:
                period_ret = (data_ma['Close'] - data_ma['Open']) \
                                / data_ma['Open'] * 100
                colors = ['r' if ret < 0 else 'g' for ret in period_ret]
                vol_bar_color = pd.Series(colors,
                                    index=period_ret.index
                                    ).loc[start:end]

            # Plot volume bars
            axs[1].bar(x=data_to_plot.index,
                        height=data_to_plot['Volume'],
                        color=vol_bar_color,
                        edgecolor=vol_bar_color,
                        label='Volume')

            # Volume Moving Average
            if volume_ma is True:
                volume_ma = 20
            if volume_ma is not False:
                axs[1].plot(data_ma['Volume'].rolling(volume_ma).mean().loc[start:end],
                            color=vol_ma_color,
                            label=f'Vol {volume_ma}MA')

            axs[1].grid(which='major', color='grey', linestyle=':')
            axs[1].set_ylabel('Volume')
            axs[1].legend(loc=legendloc)

            plt.xticks(np.arange(len(data_to_plot),
                                step=max(len(data_to_plot) // 10, 1)),
                    size=7)
            plt.xlabel('Date')

            plt.show()

    ################################
    #### Calculations
    ## sharpe_ratio()
    ## correlations()
    ## calc_rsi()
    ## calc_historical_volatility()
    ## calc_bollinger_bands()
    ## calc_macd()
    ## monthly_returns()
    ## hourly_returns()
    ################################

    def sharpe_ratio(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price:      str   = 'Close',
            risk_free_rate: float = 0.05
    ) -> float:
        """
            Calculates the Sharpe Ratio of a ticker.
            #### return float

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - risk_free_rate: float=0.05
                The Risk-Free Rate to use.

            Example use
            ---
            `sharpe_r = data.sharpe_ratio('SPY')`
        """

        data_to_use = self.loc[start:end][ticker][use_price]

        returns = data_to_use.pct_change()
        portfolio_return = returns.mean()
        portfolio_volatility = returns.std()

        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

        return sharpe_ratio

    def correlations(
            self,
            tickers:   list[str] | None = None,
            start:     str | None       = None,
            end:       str | None       = None,
            *,
            use_price: str  = 'Close',
            plot:      bool = True,
            style:     str  = 'dark_background'
    ) -> pd.DataFrame:
        """
            Calculates the correlation between assets and plots a heatmap.
            #### return pd.DataFrame

            Params
            ---
            - ticker: list[str] | None=None
                The tickers to use,
                if None, default to use all.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close_norm'
                The normalized price column to be used for plotting the price.
            - plot: bool=True
                If to show the heatmap of the correleations.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`

            Example use
            ---
            `corr_matrix = data.correlations()`
        """

        data_to_use = self.loc[:, pd.IndexSlice[:, use_price]]
        if tickers is None:
            tickers = data_to_use.columns
        data_to_use = data_to_use.loc[start:end][tickers]

        returns = data_to_use.pct_change()

        # Correlation matrix
        corr_matrix = returns.corr()
        corr_matrix.columns = corr_matrix.columns.get_level_values(0)
        corr_matrix.columns.name = None
        corr_matrix.index = corr_matrix.index.get_level_values(0)
        corr_matrix.index.name = None

        # Plot
        if plot:
            with plt.style.context(style):
                plt.title('Correlations', size=18)
                sns.heatmap(corr_matrix,
                            cmap='coolwarm',
                            center=0,
                            annot=True,
                            square=True)
                plt.show()

        return corr_matrix

    def calc_rsi(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price: str  = 'Close',
            window:    int  = 14,
            add_col:   bool = False,
            plot:      bool = True,
            style:     str  = 'dark_background',
            figsize:   tuple[int] | None = None,
            color:     str  = 'c',
            up_line:   int  = 70,
            lo_line:   int  = 30
    ) -> pd.DataFrame:
        """
            Calculates the Relative Strength Index (RSI) of a ticker and shows a plot,
            can also get the columns of the data obtained.
            #### return pd.DataFrame or PriceData

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - window: int=14
                Window to use for calculations.
            - add_col: bool=False
                If to add columns of the calculations to the source data ticker.
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - color: str='c':
                Color of the RSI line.
            - up_line: int=70
                Place of upper reference line in the plot.
            - lo_line: int=30
                Place of lower reference line in the plot.

            Example use
            ---
            `data.calc_rsi('SPY', add_col=True)`
        """

        data_to_use = self.loc[start:end][ticker][use_price]

        # Calculate RSI
        delta = data_to_use.diff(1)
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window).mean()
        avg_loss = loss.rolling(window).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        if plot:
            # Prepate data to plot
            data_to_plot = rsi.copy()
            date_format = '%Y-%m-%d'
            if data_to_plot.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            data_to_plot.index = data_to_plot.index.strftime(date_format)

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} Relative Strength Index (RSI)', size=18)

                plt.plot(data_to_plot, color=color, label='RSI')
                plt.axhline(up_line, color='r', linestyle='--', label=f'Upper  Line ({up_line})')
                plt.axhline(50     , color='b', linestyle='--', label='Middle Line (50)')
                plt.axhline(lo_line, color='g', linestyle='--', label=f'Lower  Line ({lo_line})')

                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.xticks(np.arange(len(data_to_plot),
                                     step=max(len(data_to_plot) // 10, 1)),
                        size=7)
                plt.ylim(0, 100)
                plt.legend()
                plt.xlabel('Date')
                plt.ylabel('RSI')
                plt.show()

        if add_col:
            self[(f'{ticker}', 'RSI')] = rsi
            return self
        else:
            df = pd.DataFrame()
            df['RSI'] = rsi
            return df

    def calc_historical_volatility(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price: str  = 'Close',
            window:    int  = 20,
            add_col:   bool = False,
            plot:      bool = True,
            style:     str  = 'dark_background',
            figsize:   tuple[int] | None = None,
            color:     str  = 'c'
    ) -> pd.DataFrame:
        """
            Calculates the historical volatility of a ticker and shows a plot,
            can also get the columns of the data obtained.
            #### return pd.DataFrame or PriceData

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - window: int=20
                Window to use for calculations.
            - add_col: bool=False
                If to add columns of the calculations to the source data ticker.
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - color: str='c':
                Color of the Volatility line.

            Example use
            ---
            `data.calc_historical_volatility('SPY', add_col=True)`
        """

        data_to_use = self.loc[start:end][ticker][use_price]

        log_return = np.log(data_to_use / data_to_use.shift(1))
        volatility = log_return.rolling(window).std() * np.sqrt(252)

        if plot:
            # Prepate data to plot
            data_to_plot = volatility.copy()
            date_format = '%Y-%m-%d'
            if data_to_plot.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            data_to_plot.index = data_to_plot.index.strftime(date_format)

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} Historical Volatility', size=18)

                plt.plot(data_to_plot, color=color, label='Volatility')
                plt.axhline(data_to_plot.mean(), color='y', linestyle='--', label='Average')

                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.xticks(np.arange(len(data_to_plot),
                                    step=max(len(data_to_plot) // 10, 1)),
                        size=7)
                plt.legend()
                plt.xlabel('Date')
                plt.ylabel('Volatility')
                plt.show()

        if add_col:
            self[(f'{ticker}', 'Log Return')] = log_return
            self[(f'{ticker}', 'Log Volatility')] = volatility
            return self
        else:
            df = pd.DataFrame()
            df['Log Return'] = log_return
            df['Volatility'] = volatility
            return df

    def calc_bollinger_bands(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price: str   = 'Close',
            window:    int   = 20,
            num_std:   float = 2.0,
            add_col:   bool  = False,
            plot:      bool  = True,
            style:     str   = 'dark_background',
            figsize:   tuple[int] | None = None,
            color:     str   = 'c'
    ) -> pd.DataFrame:
        """
            Calculates the Bollinger Bands of a ticker and shows a plot,
            can also get the columns of the data obtained.
            #### return pd.DataFrame or PriceData

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - window: int=20
                Window to use for calculations.
            - num_std: float=2.0
                Number of standard deviations for the upper and lower bands.
            - add_col: bool=False
                If to add columns of the calculations to the source data ticker.
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)
            - color: str='c':
                Color of the Price line.

            Example use
            ---
            `data.calc_bollinger_bands('SPY', add_col=True)`
        """

        data_to_use = self.loc[start:end][ticker][use_price]

        std = data_to_use.rolling(window).std()

        middle_band = data_to_use.rolling(window).mean()
        upper_band  = middle_band + num_std * std
        lower_band  = middle_band - num_std * std

        if plot:
            # Prepate data to plot
            data_to_plot = data_to_use
            date_format = '%Y-%m-%d'
            if data_to_plot.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            data_to_plot.index = data_to_plot.index.strftime(date_format)

            upper_band .index = upper_band .index.strftime(date_format)
            middle_band.index = middle_band.index.strftime(date_format)
            lower_band .index = lower_band .index.strftime(date_format)

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} Bollinger Bands', size=18)

                plt.plot(data_to_plot, color=color, label=f'{use_price} Price')
                plt.plot(upper_band , color='r', alpha=0.7,
                        label=f'Upper  Band ({window}MA + {num_std} STD)')
                plt.plot(middle_band, color='b', alpha=0.7,
                        label=f'Middle Band ({window}MA)')
                plt.plot(lower_band , color='g', alpha=0.7,
                        label=f'Lower  Band ({window}MA - {num_std} STD)')
                plt.fill_between(data_to_plot.index, upper_band, lower_band,
                                color='grey', alpha=0.4)

                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.xticks(np.arange(len(data_to_plot),
                                    step=max(len(data_to_plot) // 10, 1)),
                        size=7)
                plt.legend()
                plt.xlabel('Date')
                plt.ylabel('Price ($)')
                plt.show()


        if add_col:
            self[(f'{ticker}', 'Upper Band' )] = upper_band
            self[(f'{ticker}', 'Middle Band')] = middle_band
            self[(f'{ticker}', 'Lower Band' )] = lower_band
            return self
        else:
            df = pd.DataFrame()
            df['Upper Band' ] = upper_band
            df['Middle Band'] = middle_band
            df['Lower Band' ] = lower_band
            return df

    def calc_macd(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price:     str  = 'Close',
            short_window:  int  = 12,
            long_window:   int  = 26,
            signal_window: int  = 9,
            add_col:       bool = False,
            plot:          bool = True,
            style:         str  = 'dark_background',
            figsize:       tuple[int] | None = None
    ) -> pd.DataFrame:
        """
            Calculates the Moving Average Convergence Divergence (MACD) of a ticker and shows a plot,
            can also get the columns of the data obtained.
            #### return pd.DataFrame or PriceData

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - short_window: int=12
                Window to use for the short term exponential moving average calculation.
            - long_window: int=26
                Window to use for the long term exponential moving average calculation.
            - signal_window: int=9
                Window to use for the signal calculation.
            - add_col: bool=False
                If to add columns of the calculations to the source data ticker.
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)

            Example use
            ---
            `data.calc_macd('SPY', add_col=True)`
        """

        data_to_use = self.loc[start:end][ticker][use_price]

        short_ema = data_to_use.ewm(span=short_window, adjust=False).mean()
        long_ema = data_to_use.ewm(span=long_window, adjust=False).mean()
        macd = short_ema - long_ema
        signal = macd.ewm(span=signal_window, adjust=False).mean()
        macd_histogram = macd - signal

        if plot:
            # Prepate data to plot
            date_format = '%Y-%m-%d'
            if macd.index.name == 'Datetime':
                date_format = '%Y-%m-%d %H:%M'
            macd.index = macd.index.strftime(date_format)
            signal.index = signal.index.strftime(date_format)

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} MACD', size=18)

                plt.plot(macd, label='MACD', color='c')
                plt.plot(signal, label='Signal', color='r')
                plt.bar(macd.index, macd_histogram,
                        label='Histogram', color='gray', alpha=0.8)

                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.xticks(np.arange(len(macd),
                                    step=max(len(macd) // 10, 1)),
                        size=7)
                plt.legend()
                plt.xlabel('Date')
                plt.ylabel('MACD')
                plt.show()

        if add_col:
            self[(f'{ticker}', f'EMA{short_window}'    )] = short_ema
            self[(f'{ticker}', f'EMA{long_window}'     )] = long_ema
            self[(f'{ticker}', 'MACD'                  )] = macd
            self[(f'{ticker}', f'Signal{signal_window}')] = signal
            self[(f'{ticker}', 'MACD_Histogram'        )] = macd_histogram
            return self
        else:
            df = pd.DataFrame()
            df[f'EMA{short_window}'    ] = short_ema
            df[f'EMA{long_window}'     ] = long_ema
            df['MACD'                  ] = macd
            df[f'Signal{signal_window}'] = signal
            df['MACD_Histogram'        ] = macd_histogram
            return df

    def monthly_returns(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price: str  = 'Close',
            as_pct:    bool = True,
            plot:      bool = True,
            style:     str  = 'dark_background',
            figsize:   tuple[int] | None = None
    ) -> pd.DataFrame:
        """
            Calculates the Mean Monthly Returns of a ticker and shows a plot.
            #### return pd.DataFrame

            Params
            ---
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - as_pct: bool=True
                If True, values are expressed as percentages (e.g. 1.23 (%)),
                if False, as decimals (e.g. 0.0123).
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)

            Example use
            ---
            `monthly_ret = data.monthly_returns('SPY')`
        """

        data_to_use = self.loc[start:end][ticker][use_price]
        returns = data_to_use.resample('M').ffill().pct_change()

        if as_pct:
            returns *= 100
            y_axis_label = '%'
        else:
            y_axis_label = 'decimal'

        mean_ret = returns.groupby(returns.index.month).mean().dropna()

        # Prepare data to be returned
        df = pd.DataFrame()
        df['Mean Monthly Return'] = mean_ret
        df.index.name = 'Month'

        if plot:
            # Prepate data to plot
            months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            map_month = {i: month for i, month in zip(range(1, 13), months)}
            mean_ret.index = mean_ret.index.map(map_month)

            color = ['r' if i < 0 else 'g' for i in mean_ret]

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} Average Monthly Returns', size=18)

                plt.bar(mean_ret.index,
                        mean_ret,
                        color=color,
                        edgecolor='w')
                plt.axhline(0, linewidth=1)
                plt.axhline(mean_ret.mean(),
                            color='y',
                            linestyle='--',
                            label='Average')

                plt.xticks(rotation=0)
                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.legend()
                plt.xlabel('Month')
                plt.ylabel(f'Average Return ({y_axis_label})')
                plt.show()

        return df

    def hourly_returns(
            self,
            ticker: str,
            start:  str | None = None,
            end:    str | None = None,
            *,
            use_price: str  = 'Close',
            as_pct:    bool = True,
            plot:      bool = True,
            style:     str  = 'dark_background',
            figsize:   tuple[int] | None = None
    ) -> pd.DataFrame:
        """
            Calculates the Mean Hourly Returns of a ticker and shows a plot.
            #### return pd.DataFrame

            Params
            ---
            - data: pd.DataFrame
                Takes price data of interval '60m' or shorter.
            - ticker: str
                The ticker to use.
            - start: str | None=None
                The start date formatted as 'YYYY-MM-DD', inclusive.
            - end: str | None=None
                The end date formatted as 'YYYY-MM-DD', inclusive.
            - use_price: str='Close'
                The price column to use.
            - as_pct: bool=True
                If True, values are expressed as percentages (e.g. 1.23 (%)),
                if False, as decimals (e.g. 0.0123).
            - plot: bool=True
                If to show the plot.
            - style: str='dark_background'
                Style of the plot, check available styles with `plt.style.available`
            - figsize: tuple[int] | None=None
                Width and height of the plot,
                if None, defaults to (16, 6)

            Example use
            ---
            `hourly_ret = data.hourly_returns('SPY')`
        """

        data_to_use = self.loc[start:end][ticker][use_price]
        returns = data_to_use.pct_change()

        if as_pct:
            returns *= 100
            y_axis_label = '%'
        else:
            y_axis_label = 'decimal'

        mean_ret = returns.groupby(returns.index.hour).mean().dropna()
        mean_ret = mean_ret.loc[9:15]

        # Prepare data to be returned
        df = pd.DataFrame()
        df['Mean Hourly Return'] = mean_ret
        df.index.name = 'Hour'

        if plot:
            color = ['r' if i < 0 else 'g' for i in mean_ret]

            if figsize is None:
                figsize = (16, 6)

            # Plot
            with plt.style.context(style):
                plt.figure(figsize=figsize)
                plt.title(f'{ticker} Average Hourly Returns', size=18)

                plt.bar(mean_ret.index,
                        mean_ret,
                        color=color,
                        edgecolor='w')
                plt.axhline(0, linewidth=1)
                plt.axhline(mean_ret.mean(),
                            color='y',
                            linestyle='--',
                            label='Average')

                # plt.xticks(rotation=0)
                plt.grid(which='major', color='grey', zorder=0, linestyle=':')
                plt.legend()
                plt.xlabel('Hour')
                plt.ylabel(f'Average Return ({y_axis_label})')
                plt.show()

        return df


# Code for testing

## Getting data

In [3]:
# tickers = ['SPY', 'AAPL', 'TSLA', 'KO']
# data = PriceData(tickers, '2023-01-01', '2024-01-01', '1d')
# data

In [4]:
# data_ret = data.pct_returns(values={'Daily':    ('Close', 'Close', 1),
#                                     'Intraday': ('Close', 'Open' , 0),
#                                     'HL_diff':  ('High' , 'Low'  , 0)})
# data_ret.head(10)

In [5]:
# data_ret_d = data.pct_returns_daily()
# data_ret_d.head(10)

In [6]:
# data_ins = data_ret.insights(order_by='ins')
# data_ins

In [7]:
# data_ins.top_results(top=2, bot=1, round_val=3)

In [8]:
# data_norm = data.normalize_values()
# data_norm

## Drop and datetime

In [9]:
# data.drop_columns('Adj Close')

In [10]:
# data.from_datetime('weekday', add_col=True)

## Plots

In [11]:
# data.plot_price('SPY', color='w', mas=[20, 50, 100, 150])

In [12]:
# data.plot_volume('SPY', ma=50)

In [13]:
# data.plot_price_vol('SPY', price_color='w', price_mas=[20, 50, 100, 150])

In [14]:
# data_norm.plot_norm_moves(['SPY', 'AAPL'], colors=['w', 'r'])

In [15]:
# data_norm.plot_norm_vol(['SPY', 'AAPL'], colors=['w', 'r'])

In [16]:
# data_ret.plot_dist_ret(['SPY', 'AAPL'], colors=['w', 'r'], use_ret='Intraday')

## Calcs

In [17]:
# sharpe_r = data.sharpe_ratio('SPY')
# sharpe_r

In [18]:
# corr_matrix = data.correlations()
# corr_matrix

In [19]:
# data.calc_rsi('SPY', add_col=False)

In [20]:
# data.calc_historical_volatility('SPY', add_col=True)

In [21]:
# data.calc_bollinger_bands('SPY', add_col=True)

In [22]:
# data.calc_macd('SPY', add_col=True)

In [23]:
# monthly_ret = data.monthly_returns('SPY')
# monthly_ret

In [24]:
# data = PriceData(tickers, '2022-07-01', '2024-05-01', '60m')
# hourly_ret = data.hourly_returns('SPY')
# hourly_ret